In [3]:
#Подключение библиотек для работы с файлами
import pandas as pd 
import numpy as np

In [2]:
#Чтение основного файла 
df_train = pd.read_csv('train.csv')

In [65]:
#Ограничение для комфортной работы, необязательно если достаточно RAM 
df_train = df_train[:1500000]

In [66]:
#Список уникальных юнитов 
#Изначально искал их так: list_of_units = df_train['X5'].unique(), выяснив что всего 101 юнит, просто стал создавать лист 
list_of_units = [i for i in range(101)] 



## Саздание таблицы признаков

In [67]:
#Функция для k-hot encoding, используется для перекодирование таблиц с юнитами в бинарный набор, 
#где 1 - наличие в наборе юнита с таким номером, 0 - отсутствие
def k_hot_encoder(data_frame_of_values, list_of_category):
    K_hot_encoder_set = (data_frame_of_values.values[:,:,None] == list_of_category).any(1).astype(int)
    return pd.DataFrame(K_hot_encoder_set, columns=list_of_category)

In [68]:
def preprocessing_data(data_features):
    
    #Кодирование переменной 'X0'
    X = pd.get_dummies(data_features['X0'] ,prefix='X0')
    
    #Добавление комбинаций юнитов для первого игрока 
    Xh1 = k_hot_encoder(data_features[['X5', 'X6', 'X7', 'X8', 'X9', 'X10','X11', 'X12']], list_of_units)#кодирование
    #Добавление комбинаций юнитов для второго игрока 
    X = X.join(Xh1, rsuffix='first_set')    
    
    #Кодирование и добавление комбинаций юнитов для второго игрока в обучающий датасет
    Xh2 = k_hot_encoder(data_features[['X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20']], list_of_units) #кодирование
    X = X.join(Xh2, rsuffix='second_set')
    
    #Переменная для сравнение рейтингов игроков 
    X['compare_rang_players'] = np.where(data_features['X2'] > data_features['X4'], 1, 0)
    return X

In [69]:
#Таблица с признаками
X = preprocessing_data(df_train)

In [71]:
#Переменная предсказаний
y = df_train['target']

In [73]:
#Использовал для проверки
#Составление датасетов для валидации 
#from sklearn.model_selection import train_test_split
#X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.30, random_state=7)
#X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

## Модель предсказания победителя


In [74]:
#Основана на методе градиентного бустинга  

In [75]:
#Реализация классификации при помощи градиентного бустинга от Яндекса, подключаем библиотеку catboost и нужный модуль
from catboost import CatBoostClassifier

In [78]:
#Создаем модель с параметрами
model = CatBoostClassifier(  
               iterations=5300,
               learning_rate=0.11,
               task_type="GPU",
               verbose = False   
)

In [2]:
#Обучаем модель
model.fit(
    X, y
    #X_train, y_train использовал для валидации 
)

NameError: name 'model' is not defined

In [1]:
#Использовал для валидации
#from sklearn.metrics import log_loss
#log_loss(y_valid,y_pred)

## Применение модели для тестовых данных (test.csv)

In [3]:
#Чтение тестовых данных 
df_test = pd.read_csv('test.csv')

NameError: name 'pd' is not defined

In [ ]:
#Препроцессинг для тестовых данных
df_test = preprocessing_data(df_test)

In [ ]:
#Прогноз модели для тестовых данных
y_pred = model.predict_proba(df_test)

In [ ]:
#Создание датафрейма [id,предсказание] и cохранение в файл  
rez = pd.DataFrame({'id':df_test['id'], 'target':y_pred[:,1]},  )
rez.to_csv('submit.csv',index = False)

In [ ]:
###########################################################################################
###########################################################################################
###############НЕ входит в основное решение################################################

## Пробовал, но не давало результата, либо незначительный 

In [ ]:

#Хотел найти среди target = 0 ничьи, тк заметил, что таргет = 1 это 1/3 набора, а таргер = 0 2/3, 
#скорее всего половина с таргетом 0 это ничьи,
#Хотел найти признаки для ничьх и сделать тройную классификацию, либо методом стекинга с перераспределением весов
#Пробовал отзеркалить победы. Взять где таргет = 1, переставить игроков, их рейтинги и наборы юнитов местами,поставить 0 
#,cложить эти два датасета и обучить на этом наборе 

In [ ]:
#Делал все в colab от Google, поэтому еще подключал такой модуль
#from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)

In [ ]:
#Добавление колонок наборов игроков как листов и колонки с листом различий в наборах
a = (df_train[['X5', 'X6', 'X7', 'X8', 'X9', 'X10','X11', 'X12']].values.tolist())
b = (df_train[['X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20']].values.tolist())
s = []
for i in range(len(a)):
    s.append( list(set(a[i]) - set(b[i])) + list(set(b[i])-set(a[i])) )
df_train['dt1'] = a
df_train['dt2'] = b
df_train['delta'] = s



In [ ]:
#Столбец с наибольшими и наименьшими рангами игроков
rang_of_1_player = df_train.groupby('X1').agg({'X2':['min','max']}) #as_index = False , suffixes=('', '2')
df_train = df_train.merge(rang_of_1_player[[('X2','min')]], how='left', left_on = 'X1',  right_index = True )
df_train = df_train.merge(rang_of_1_player[[('X2','max')]], how='left', left_on = 'X1', right_index = True )

rang_of_2_player = df_train.groupby('X3').agg({'X4':['min','max']}) #as_index = False , suffixes=('', '2')
df_train = df_train.merge(rang_of_2_player[[('X4','min')]], how='left', left_on = 'X3',  right_index = True )
df_train = df_train.merge(rang_of_2_player[[('X4','max')]], how='left', left_on = 'X3', right_index = True )

In [ ]:
#Искал винрейт для игрока (количесвто побед/количество игр)
#, брал только тех, у кого 100 боев и больше, иначе игра 1/1 (и подобные) оценивалась как 100% побед 
#Винрейт игроков 
win_player = df_train[['X1','target']].groupby('X1').agg({'target':['count','sum']}) #as_index = False
win_player['winrate'] = win_player[('target','sum')] / win_player[('target','count')] 
tabwinrate = win_player[(win_player[('target','count')]>100) & (win_player[('target','count')]<300)].sort_values([('winrate')], ascending=False )
#tabwinrate
#Для записи фичи в трейн дата
df_train = df_train.merge(tabwinrate['winrate'], how='left', left_on = 'X1',  right_index = True )
df_train = df_train.merge(tabwinrate['winrate'], how='left', left_on = 'X3', right_index = True, suffixes=('', '2') )
df_train['winrate'] = df_train['winrate'].fillna(df_train['winrate'].mean(axis=0)-0.1)
df_train['winrate2'] = df_train['winrate2'].fillna(df_train['winrate2'].mean(axis=0)-0.1)
#Для записи фичи в тест дата
df_test = df_test.merge(tabwinrate['winrate'], how='left', left_on = 'X1',  right_index = True )
df_test = df_test.merge(tabwinrate['winrate'], how='left', left_on = 'X3', right_index = True, suffixes=('', '2') )
df_test['winrate'] = df_test['winrate'].fillna(df_test['winrate'].mean(axis=0)-0.1)
df_test['winrate2'] = df_test['winrate2'].fillna(df_test['winrate2'].mean(axis=0)-0.1)

In [ ]:
#Искал победные комбинации,комбинация (кол-во побед / кол-во боев), отбирал такие, которые учавсвовали в 500+ боях, других заменял на среднее  
a = list(map(frozenset, a))
df_train['set_of_first'] = a
b = list(map(frozenset, b))
df_train['set_of_second'] = b

table_of_df_train = df_train[['set_of_first','target']].groupby('set_of_first').agg({'target':['count','sum']})
table_of_df_train = table_of_df_train.sort_values([('target','sum')])
table_of_df_train[('target','deltarget')] = table_of_df_train[('target','sum')] / table_of_df_train[('target','count')] 
A = table_of_df_train[table_of_df_train[('target','count')]>500].sort_values([('target','deltarget')], ascending=False )

#для записи фичи в трейн дата
df_train = df_train.merge(A[[('target','deltarget')]], how='left', left_on = 'set_of_first',  right_index = True )
df_train = df_train.merge(A[[('target','deltarget')]], how='left', left_on = 'set_of_second',  right_index = True )
df_train["('target', 'deltarget')_x"] = df_train["('target', 'deltarget')_x"].fillna(df_train["('target', 'deltarget')_x"].mean(axis=0) - 0.2)
df_train[('target_y', 'deltarget')] = df_train[('target_y', 'deltarget')].fillna(df_train[('target_y', 'deltarget')].mean(axis=0) - 0.2)
#для записи фичи в тестовые данные
df_test = df_test.merge(A[[('target','deltarget')]], how='left', left_on = 'set_of_first',  right_index = True )
df_test = df_test.merge(A[[('target','deltarget')]], how='left', left_on = 'set_of_second',  right_index = True )
df_test["('target', 'deltarget')_x"] = df_test["('target', 'deltarget')_x"].fillna(df_test["('target', 'deltarget')_x"].mean(axis=0) - 0.2)
df_test[('target_y', 'deltarget')] = df_test[('target_y', 'deltarget')].fillna(df_test[('target_y', 'deltarget')].mean(axis=0) - 0.2)

In [ ]:
#Поиск сочитаний в наборах, определение винрейта юнита
#Для боя каждый юнит из первого(выйграшного) набора сопоставлялся с каждым юнитов во втором(проигравшем) наборе
B = df_train[df_train['target']==1]


с = B[['dt1','dt2']].values.tolist()
m_zeros = np.zeros((101, 101)) #Матрица побед
m_zeros2 = np.zeros((101, 101)) # Матрица поражений

#заполнение матриц
for i in с: 
    un1 = i[0]
    un2 = i[1]
    for j in un1:
        for k in un2:
            m_zeros[j,k] +=1
            m_zeros2[k,j] +=1   

#матрица сумм 
result = np.zeros((101, 101))
for i in range(len(m_zeros)): 
    for j in range(len(m_zeros[0])):  
              result[i][j] = m_zeros[i][j] + m_zeros2[i][j] 
              
#Матрица winrate для юнитов
result2 = np.zeros((101, 101))
for i in range(len(m_zeros)): 
    for j in range(len(m_zeros[0])):  
              result2[i][j] = m_zeros[i][j] / result[i][j] 


#Запись в трейн
c = df_train[['dt1','dt2']].values.tolist()
slist = []
s2list = []
for i in c: 
    un1 = i[0]
    un2 = i[1]
    s = 0
    s2 = 0
    for j in un1:
        for k in un2:
            s = s + result2[j,k]
            s2 = s2 + result2[k,j]
        slist.append(s)
        s2list.append(s2)
#Добавление фичи разница сумм винрейтов юниов в наборах
df_train['X22'] = [slist[i]-s2list[i] for i in range(len(slist))]





#Запись в тест
a = (df_test[['X5', 'X6', 'X7', 'X8', 'X9', 'X10','X11', 'X12']].values.tolist())
b = (df_test[['X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20']].values.tolist())
s = []
for i in range(len(a)):
    s.append( list(set(a[i]) - set(b[i])) + list(set(b[i])-set(a[i])) )
df_test['dt1'] = a
df_test['dt2'] = b
df_test['delta'] = s
c = df_test[['dt1','dt2']].values.tolist()
slist = []
s2list = []
for i in c: 
    un1 = i[0]
    un2 = i[1]
    s = 0
    s2 = 0
    for j in un1:
        for k in un2:
            s = s + result2[j,k]
            s2 = s2 + result2[k,j]
        slist.append(s)
        s2list.append(s2)
#Добавление фичи разница сумм винрейтов юниов в наборах
df_test['X22'] = [slist[i]-s2list[i] for i in range(len(slist))]



#Пытался делать винрейты для юнитов, но штрафуя(-1) за нахождение в проиграшном наборе, и +1 за нахождение в победном.
#Получалась хорошая диагональная матрица,но результат был очень не значительным, не удалось потестить на разных параметрах 

In [ ]:
#для фичей которые не получились
X = X.join(df_train[['winrate','winrate2',"('X2','max')","('X2','min')","'X4','min'","'X4','max'",'X22',('target_y', 'deltarget'),"('target', 'deltarget')_x"]])
X.drop('delta','dt1','dt2')

In [ ]:
#Также при анализе строил графики распределения для разных столбцов и подгрупп